<a href="https://colab.research.google.com/github/sokrypton/ColabDesign/blob/main/af/examples/afdesign_hotspot_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AfDesign - binder + hotspot test

In [ ]:
#@title install
%%bash
if [ ! -d params ]; then
  pip -q install git+https://github.com/sokrypton/ColabDesign.git
  mkdir params
  curl -fsSL https://storage.googleapis.com/alphafold/alphafold_params_2022-03-02.tar | tar x -C params
  for W in openfold_model_ptm_1 openfold_model_ptm_2 openfold_model_no_templ_ptm_1
  do wget -qnc https://files.ipd.uw.edu/krypton/openfold/${W}.npz -P params; done
fi

In [ ]:
#@title import libraries
import os
from colabdesign import mk_afdesign_model, clear_mem
from IPython.display import HTML
from google.colab import files
import numpy as np

#########################
def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  else:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"

In [ ]:
%%bash
wget -qnc https://raw.githubusercontent.com/casperg92/MaSIF_colab/main/example/monomerexample.pdb

In [ ]:
clear_mem()
model = mk_afdesign_model(protocol="binder")

# pos = define positions (hotspot) on target you want to target for binder
model.prep_inputs(pdb_filename="monomerexample.pdb", chain="A",
                  binder_len=20,
                  pos="33,17,6,34,30,19,37,15") 

print("target_length",model._target_len)
print("binder_length",model._binder_len)
print("weights",model.opt["weights"])

In [ ]:
model.restart(mode="soft_gumbel")
model.set_weights(con=0.0, i_con=1.0)
model.design_3stage(100, 100, 10)

In [ ]:
HTML(model.animate())

In [ ]:
model.get_seqs()

In [ ]:
model.plot_pdb()
model.save_pdb(f"{model.protocol}.pdb")

as a control, let's try hallucinate a binder without defining a hotspot

In [ ]:
clear_mem()
model = mk_afdesign_model(protocol="binder")
model.prep_inputs(pdb_filename="monomerexample.pdb", chain="A", binder_len=20)

In [ ]:
model.restart(mode="soft_gumbel")
model.design_3stage(100, 100, 10)

In [ ]:
HTML(model.animate())

In [ ]:
model.get_seqs()

In [ ]:
model.plot_pdb()
model.save_pdb(f"{model.protocol}.control.pdb")